#### k(T)

In [3]:
def read_data_kT(variant=1):
    data_tab = []
    with open('data/kT.txt') as file:
        line = file.readline()
        while line != '':
            line = line.split()
            if variant == 1:
                data_tab.append([int(line[1]), float(line[2])])
            else:
                data_tab.append([int(line[1]), float(line[3])])

            line = file.readline()

    return data_tab
    
import numpy as np

def getkFunc(variantkT):
    data_kT = read_data_kT(variantkT)
    Tvals = [np.log(data_kT[i][0]) for i in range(len(data_kT))]
    kvals = [np.log(data_kT[i][1]) for i in range(len(data_kT))]
    def k(T):
        return np.exp(np.interp([np.log(T)], Tvals, kvals)[0])
    return k


### Метод Рунге-Кутты 2-го и 4-го порядка

In [25]:
# Рунге-Кутта 2
def RungeKutta2Step(duFunc, dFFunc, ri, ui, Fi, h, alpha):
    du, dF = duFunc, dFFunc
    k1 = du(ri, ui, Fi)
    q1 = dF(ri, ui, Fi)

    h_2a = h / (2 * alpha)
    k2 = du(ri + h_2a, ui + h_2a * k1, Fi + h_2a * q1)
    q2 = dF(ri + h_2a, ui + h_2a * k1, Fi + h_2a * q1)

    yip1 = ui + h * ( (1 - alpha) * k1 + alpha * k2)
    zip1 = Fi + h * ( (1 - alpha) * q1 + alpha * q2)
    return yip1, zip1

def RungeKutta2(duFunc, dFFunc, r0, y0, z0, h, rTarget, alpha=1):
    rvals = [r0]
    yvals = [y0]
    Fvals = [z0]
    while rvals[-1] < rTarget:
        ri = rvals[-1] + h
        yi, Fi = RungeKutta2Step(duFunc, dFFunc, rvals[-1], yvals[-1], Fvals[-1], h, alpha)
        rvals.append(ri)
        yvals.append(yi)
        Fvals.append(Fi)
    return rvals, yvals, Fvals

# Рунге-Кутта 4
def RungeKutta4Step(duFunc, dFFunc, ri, ui, Fi, h):
    du, dF = duFunc, dFFunc
    k1 = du(ri, ui, Fi)
    q1 = dF(ri, ui, Fi)
    k2 = du(ri + h / 2, ui + h / 2 * k1, Fi + h / 2 * q1)
    q2 = dF(ri + h / 2, ui + h / 2 * k1, Fi + h / 2 * q1)
    k3 = du(ri + h / 2, ui + h / 2 * k2, Fi + h / 2 * q2)
    q3 = dF(ri + h / 2, ui + h / 2 * k2, Fi + h / 2 * q2)
    k4 = du(ri + h, ui + h * k3, Fi + h * q3)
    q4 = dF(ri + h, ui + h * k3, Fi + h * q3)
    yip1 = ui + (h / 6) * ( k1 + 2 * k2 + 2 * k3 + k4)
    zip1 = Fi + (h / 6) * ( q1 + 2 * q2 + 2 * q3 + q4)
    if yip1 < 0:
        print(f"RK4step: yip1={yip1}, k1={k1} k2={k2} k3={k3} k4={k4}, ri={ri}, ui={ui}, Fi={Fi}")
    return yip1, zip1

def RungeKutta4(duFunc, dFFunc, r0, y0, z0, h, rTarget):
    rvals = [float(r0)]
    yvals = [float(y0)]
    Fvals = [float(z0)]
    while rvals[-1] < rTarget:
        ri = rvals[-1] + h
        yi, Fi = RungeKutta4Step(duFunc, dFFunc, rvals[-1], yvals[-1], Fvals[-1], h)
        rvals.append(float(ri))
        yvals.append(float(yi))
        Fvals.append(float(Fi))
    return rvals, yvals, Fvals

### T(r), Up(r)

In [26]:
c = 3 * 1e10    # Скорость света
R = 0.35        # Радиус цилиндра

# температурное поле в цилиндре
def T_r(r):
    Tw, To, p = 2000, 1e4, 4
    return (Tw - To) * ((r / R) ** p) + To

# функция Планка
def up_r(r):
    return (3.084 * 1e-4) / (np.exp(4.799 * 1e4 / T_r(r)) - 1)


In [ ]:
k = getkFunc(variantkT=1)

r0 = 0
def get_y0(khi):
    return khi * up_r(r0)
z0 = 0
rTarget = R

def du(ri, ui, Fi):
    return -(3 * k(T_r(ri)) * Fi) / c

def dF(ri, ui, Fi):
    if ri == 0 and Fi == 0:
        return 0.5 * k(T_r(ri)) * (up_r(ri) - ui)
    return -Fi / ri + c * k(T_r(ri)) * (up_r(ri) - ui)

# Метод стрельбы, решение для khi
def solutionShooting(khi, RungeKuttaFunc, h_start):
    RungeKuttaEPS = 1e-5
    y0 = get_y0(khi)
    h = h_start

    # print(f"khi={khi}, h={h}, y0={y0}")
    res_h = RungeKuttaFunc(du, dF, r0, y0, z0, h, rTarget)
    yR_h = res_h[1][-1]
    # if yR_h < 0:
    #     print(f"khi={khi}, h={h}, y={res_h[1]}")
    res_h2 = RungeKuttaFunc(du, dF, r0, y0, z0, h / 2, rTarget)
    yR_h2 = res_h2[1][-1]
    # if yR_h2 < 0:
    #     print(f"khi={khi}, h={h}, y={res_h2[1]}")
    while abs(yR_h - yR_h2) / yR_h2 >= RungeKuttaEPS:
        print(f"khi = {khi}, h = {h}, yR_h = {yR_h}, yR_h2 = {yR_h2}, acc = {abs(yR_h - yR_h2) / yR_h2}")
        h /= 2
        res_h, yR_h = res_h2[:], yR_h2
        res_h2 = RungeKuttaFunc(du, dF, r0, y0, z0, h / 2, rTarget)
        # if yR_h2 < 0:
        #     print(f"khi={khi}, h={h}, y={res_h2[1]}")
        yR_h2 = res_h2[1][-1]
    
    res_yR, res_FR = res_h[1][-1], res_h[2][-1]
    return (res_yR, res_FR), h

def equationTarget(uR, FR):
    return FR - 0.39 * c * uR

# Метод половинного деления (поиск подходящего khi)
def HalfDivMethod(RungeKuttaFunc, h_start):
    khiEPS = 1e-4
    khi_l, khi_r = 1e-1, 1
    h = h_start
    res_l, h = solutionShooting(khi_l, RungeKuttaFunc, h)
    res_r, h = solutionShooting(khi_r, RungeKuttaFunc, h)

    while abs(khi_r - khi_l) / khi_l >= khiEPS:
        khi_mid = (khi_l + khi_r) / 2
        res_mid, h = solutionShooting(khi_mid, RungeKuttaFunc, h)
        if equationTarget(*res_mid) * equationTarget(*res_l) < 0:
            khi_r = khi_mid
        else:
            khi_l = khi_mid
        print(f"khi = {khi_mid}, yR = {res_mid[0]}, FR = {res_mid[1]}, h = {h}")
    print(f"\nSolution: khi = {khi_mid}, yR = {res_mid[0]}, FR = {res_mid[1]}, h = {h}")
    return khi_mid, h

h_st = 1e-2
khi_solution, h_solution = HalfDivMethod(RungeKutta4, h_st)



khi=0.1, h=0.01, y0=2.5616965203959684e-07
khi = 0.1, h = 0.01, yR_h = 1.64140055995201e-07, yR_h2 = 1.6409265280565374e-07, acc = 0.0002888806338173393
khi = 0.1, h = 0.005, yR_h = 1.6409265280565374e-07, yR_h2 = 1.6408078952588673e-07, acc = 7.23014546753904e-05
khi = 0.1, h = 0.0025, yR_h = 1.6408078952588673e-07, yR_h2 = 1.6407373911135775e-07, acc = 4.297101149256109e-05
khi=1, h=0.00125, y0=2.561696520395968e-06
khi=0.55, h=0.00125, y0=1.4089330862177825e-06
khi = 0.55, yR = 1.3689068961132813e-06, FR = 743.8470531075526, h = 0.00125
khi=0.325, h=0.00125, y0=8.325513691286897e-07
khi = 0.325, yR = 7.664903176123192e-07, FR = 2360.5023308747554, h = 0.00125
khi=0.21250000000000002, h=0.00125, y0=5.443605105841433e-07
khi = 0.21250000000000002, yR = 4.652820283618393e-07, FR = 3168.829969758359, h = 0.00125
khi=0.15625, h=0.00125, y0=4.0026508131187e-07
khi = 0.15625, yR = 3.146778837365979e-07, FR = 3572.9937892001617, h = 0.00125
khi=0.128125, h=0.00125, y0=3.282173666757334e-07


In [ ]:
import plotly.express as px
import pandas as pd

RungeKutta = RungeKutta4
y0 = get_y0(khi_solution)
print(f"khi = {khi_solution}, h = {h_solution}")
rvals, yvals, Fvals = RungeKutta(du, dF, r0, y0, z0, h_solution, rTarget)
upvals = [up_r(ri) for ri in rvals]

df1 = pd.DataFrame({"r": rvals, "f": yvals, "label": ["u(r)"] * len(rvals) })
df2 = pd.DataFrame({"r": rvals, "f": Fvals, "label": ["F(r)"] * len(rvals)})
df3 = pd.DataFrame({"r": rvals, "f": upvals, "label": ["up(r)"] * len(rvals)})

df = pd.concat([df1, df3])
fig2 = px.line(df, x="r", y="f", color="label", title="", markers = True)
fig2.show()

fig1 = px.line(df2, x="r", y="f", color="label", title="", markers = True)
fig1.show()

khi = 0.15343475341796878, h = 0.00125


In [ ]:
import plotly.express as px
import pandas as pd

vkT = 2
h = 0.00125

def draw_Tr():
    rvals = [i * h for i in range(int(R // h))]
    Tvals = [T_r(r) for r in rvals]

    df1 = pd.DataFrame({"r": rvals, "T": Tvals, "label": ["T_r(r)"] * len(rvals) })
    fig = px.line(df1, x="r", y="T", color="label", title="T_r(r)", markers = True)
    fig.show()

draw_Tr()

def draw_kTr(variantkT=vkT):
    k = getkFunc(variantkT)
    rvals = [i * h for i in range(int(R // h))]
    Tvals = [T_r(r) for r in rvals]
    kvals = [k(t) for t in Tvals]

    df1 = pd.DataFrame({"T": Tvals, "k(T)": kvals, "label": [f"Newton 1"] * len(Tvals) })
    df = df1

    fig = px.line(df, x="T", y="k(T)", color="label", title=f"k(T) v{variantkT}", markers = True)
    fig.show()

draw_kTr()